In [ ]:
from datetime import datetime, timedelta
from pathlib import Path
import requests
from urllib.parse import urlparse
import zipfile
import os
import cx_Oracle
import pandas as pd


username = 'trading'
password = 'trading'
dsn = 'localhost:1521/XE'
table_name = 'TRADES'

# Function to generate working day dates between two dates
def generate_working_day_urls(start_date, end_date):
    current_date = start_date
    urls = []

    while current_date <= end_date:
        if current_date.weekday() < 5:  # Weekdays (Monday to Friday)
            formatted_date = current_date.strftime("%Y/%b/%d").upper()
            year, month, day = formatted_date.split('/')
            url = f"https://archives.nseindia.com/content/historical/EQUITIES/{year}/{month}/cm{day}{month}{year}bhav.csv.zip"
            urls.append(url)
        current_date += timedelta(days=1)

    return urls
# End of function generate_working_day_urls

# Function to get filename from the URL
def get_filename_from_url(url):
    # Parse the URL
    parsed_url = urlparse(url)
    # Extract the path component and split it to get the file name
    file_name = parsed_url.path.split('/')[-1]
    return file_name
# End of function get_filename_from_url

# Function to unzip the downloaded Bhavcopy zip file
def unzip_file(zip_file_path, extract_to_folder=None):
    if extract_to_folder is None:
        extract_to_folder = os.path.dirname(zip_file_path)

    # Extract the zip file contents
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to_folder)
        print(f"Extracted all files in {zip_file_path}")
# End of function unzip_file

# Function to insert the data to DB
def insert_data(data, table_name, username, password, dsn):
    # Establish a database connection
    connection = cx_Oracle.connect(username, password, dsn)
    cursor = connection.cursor()

    # Calculate the percentage change
    data["CHANGE_PCT"] = round(((data["CLOSE"] - data["PREVCLOSE"]) / data["PREVCLOSE"]) * 100, 2)

    # Prepare insert query
    placeholders = ', '.join([':' + str(i + 1) for i in range(len(data.columns))])
    query = f"INSERT INTO {table_name}  VALUES ({placeholders})"

    # Get the data to be inserted
    rows = [tuple(x) for x in data.values]

    # Execute insert statement
    cursor.executemany(query, rows)

    # Commit the transaction and close the connection
    connection.commit()
    cursor.close()
    connection.close()
    print("Data inserted successfully. \n")
# End of function insert_data
    

# Main Processing

# Define start and end dates (YYYY, MM, DD)
start_date = datetime(2024, 2, 1)
end_date = datetime.today()


# Generate URLs
urls = generate_working_day_urls(start_date, end_date)

# Retreive Bhavcopy zip file from the URLs and insert to DB
for url in urls:
    response = requests.get(url, timeout=5)
    if response.status_code == 200:
        # Write the content of the response to a local file
        with open(get_filename_from_url(url), 'wb') as file:
            file.write(response.content)
            unzip_file(get_filename_from_url(url))
            print("Downloaded: " + get_filename_from_url(url))
            tradeData = pd.read_csv(Path(get_filename_from_url(url)).stem,  keep_default_na=False, na_values=['_'])
            tradeData.drop(0, axis=0, inplace=True)
            tradeData.drop(columns=tradeData.columns[-1],  axis=1,  inplace=True)
            print("Processing: " + get_filename_from_url(url))

            # Save data in DB
            insert_data(tradeData, table_name, username, password, dsn)
    else:
        print(f"Markets are closed due to public holiday")